## Задание 2. Распилы льда

#### Описание задачи:

Большинство метеонаблюдений до сих пор сначала заносится в тетрадку, а потом передаётся по рации. В результате после оцифровки в датасете о распилах льда на реке Лена оказываются ошибки. Где они и как с ними бороться?

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Чтение данных

# Список листов excel
sheets = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# Заготовка датафрейма
df = pd.DataFrame()

# Постраничное чтение excel в цикле, добавление отметки о годе и добавление в итоговый датафрейм
for sheet in sheets:
    new_sheet = pd.read_excel('Мероприятия для МЧС (1803 - Лена).xlsx', sheet_name=sheet, header=[0,1])
    new_sheet['Год'] = sheet
    df = df.append(new_sheet)
    
df.head()

Расположение                Тип  \
  Unnamed: 0_level_1 Unnamed: 1_level_1   
0              Ленск               СУБВ   
1              Ленск               СУБВ   
2             Якутск               СУБВ   
3           Покровск               СУБВ   
4          Олёкминск                ГТС   

                           Наименование мероприятия        Координаты  \
                                  Unnamed: 2_level_1 Широта, Градусов   
0  Ослабление прочности льда на р.Нюя в Ленском р...               61   
1  Определение границ водоохранной зоны и прибреж...               35   
2  Определение границ водоохранной зоны и прибреж...               62   
3  Определение границ водоохранной зоны и прибреж...                0   
4  Ослабление прочности льда на р.Лена в Олекминс...               60   

                                                                 \
  Широта, Минут Широта, Секунд Долгота, Градусов Долгота, Минут   
0            22             42               114             33   
1            43             60               114             56   
2             1             38               129             43   
3            29             61               129              9   
4            23              0               120             26   

                    Влияние на водный объект  \
  Долгота, Секунд         Unnamed: 9_level_1   
0              54  Ослабление прочности льда   
1              43                Нет влияния   
2              55                Нет влияния   
3               3                Нет влияния   
4               0  Ослабление прочности льда   

                                                         Год  
  СУБВ - мероприятие осуществляется за счет субвенций         
0                        ГТС - основная деятельность    2011  
1                                                NaN    2011  
2                                                NaN    2011  
3                                                NaN    2011  
4                                                NaN    2011

In [3]:
# Приведение названий столбцов к единому формату
df.rename(columns={'Unnamed: 0_level_1': 'Расположение', 'Unnamed: 1_level_1': 'Тип', 'Unnamed: 2_level_1':
                  'Наименование мероприятия', 'Unnamed: 9_level_1': 'Влияние на водный объект', '': 'Год'},
         inplace=True)
df.columns = df.columns.droplevel(0)

df.head()

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,СУБВ - мероприятие осуществляется за счет субвенций,Год
0,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61,22,42,114,33,54,Ослабление прочности льда,ГТС - основная деятельность,2011
1,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35,43,60,114,56,43,Нет влияния,NaN,2011
2,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62,1,38,129,43,55,Нет влияния,NaN,2011
3,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0,29,61,129,9,3,Нет влияния,NaN,2011
4,Олёкминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60,23,0,120,26,0,Ослабление прочности льда,NaN,2011


In [4]:
# Общая информация по датафрейму: размеры, типы данных, пропущенные значения
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110 entries, 0 to 7
Data columns (total 12 columns):
 #   Column                                                Non-Null Count  Dtype 
---  ------                                                --------------  ----- 
 0   Расположение                                          110 non-null    object
 1   Тип                                                   110 non-null    object
 2   Наименование мероприятия                              110 non-null    object
 3   Широта, Градусов                                      110 non-null    object
 4   Широта, Минут                                         110 non-null    object
 5   Широта, Секунд                                        110 non-null    object
 6   Долгота, Градусов                                     110 non-null    object
 7   Долгота, Минут                                        110 non-null    object
 8   Долгота, Секунд                                       110 non-null    ob

In [5]:
# Зададим индексы строк для удобства
df['index'] = range(110)
df.set_index('index', drop=True, inplace=True)
df.head()

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,СУБВ - мероприятие осуществляется за счет субвенций,Год
index,,,,,,,,,,,,
0,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61,22,42,114,33,54,Ослабление прочности льда,ГТС - основная деятельность,2011
1,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35,43,60,114,56,43,Нет влияния,NaN,2011
2,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62,1,38,129,43,55,Нет влияния,NaN,2011
3,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0,29,61,129,9,3,Нет влияния,NaN,2011
4,Олёкминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60,23,0,120,26,0,Ослабление прочности льда,NaN,2011


In [6]:
# удаление лишнего столбца
df.drop(df.columns[-2], axis=1, inplace=True)
df.head()

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год
index,,,,,,,,,,,
0,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61,22,42,114,33,54,Ослабление прочности льда,2011
1,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35,43,60,114,56,43,Нет влияния,2011
2,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62,1,38,129,43,55,Нет влияния,2011
3,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0,29,61,129,9,3,Нет влияния,2011
4,Олёкминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60,23,0,120,26,0,Ослабление прочности льда,2011


### Промежуточные выводы

1. Все данные имеют тип *object*. Для удобства нужно будет преобразовать часть из них в числовые типы


2. Предпоследний столбец можно удалить из датафрейма. Если обратиться к исходному excel-файлу, можно увидеть, что данный столбец является просто расшифровкой значений для столбца "тип".


3. В столбце **"Влияние на водный объект"** есть пропущенные значения. 


4. В датафрейме много текстовых столбцов. При ручном вводе там могли оказаться опечатки. Это необходимо проверить


5. (дополнительно опционально) Столбцы названы по-русски. Это может быть неудобно для аналитики в python, так как постоянно придется переключать раскладку клавиатуры. Для удобства можно было бы переименовать столбцы на английские названия 

### Шаг 1. Работа с ошибками текстовых данных

In [7]:
# В текстовые столбцы могли попасть опечатки. Проверим все уникальные значения в таких столбцах
df['Расположение'].unique()

array(['Ленск', 'Якутск', 'Покровск', 'Олёкминск', 'Саха (Якутия)',
       'Якутстк', 'Олекминск', 'Сангар', 'г. Киренск', 'Саха Якутия'],
      dtype=object)

In [8]:
# избавимся от опечаток и приведем города к единообразию

# Саха (Якутия) тоже приведем к единому виду, однако надо понимать, что здесь все равно остается ошибка, 
# так как это не город, а субъект. Можно будет попытаться восстановить конкретный населенный пункт по описанию 
# мероприятия

# словарь для замены
errors_dict = {'Якутстк': 'Якутск'
               , 'Олёкминск': 'Олекминск'
               , 'г. Киренск': 'Киренск'
               , 'Саха Якутия': 'Саха (Якутия)'}
# замена
df = df.replace({'Расположение': errors_dict})

# проверка результата
df['Расположение'].unique()

array(['Ленск', 'Якутск', 'Покровск', 'Олекминск', 'Саха (Якутия)',
       'Сангар', 'Киренск'], dtype=object)

In [9]:
df['Тип'].unique()

array(['СУБВ', 'ГТС'], dtype=object)

In [10]:
df['Влияние на водный объект'].unique()

array(['Ослабление прочности льда', 'Нет влияния', nan], dtype=object)

In [11]:
# количество уникальных значений наименований мероприятий
len(df['Наименование мероприятия'].unique())

77

In [12]:
# Проверим, верно ли указаны города в столбце "Расположение"

# Вытаскиваем из наименования мероприятия местоположение

# Регулярное выражение для поиска названий района (вхождения от предлога "в" до заглавной буквы Р 
# (=Республика или Республики и т.д.))
reg_region = re.compile(r'в ([^Р]*)')

strings = []

# построчная итерация датафрейма для работы регулярки
for row in df.iterrows():
    string = re.findall(reg_region, row[1][2])
    # исключения для строк, в которых регулярка вернет пустой список
    try:
        strings.append(string[0])
    except:
        strings.append(0)

# новый столбец датафрейма
df['Расположение_верно'] = strings
df.head()

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,
0,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61,22,42,114,33,54,Ослабление прочности льда,2011,"Ленском районе,"
1,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35,43,60,114,56,43,Нет влияния,2011,"районе г. Ленска,"
2,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62,1,38,129,43,55,Нет влияния,2011,"районе г. Якутска,"
3,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0,29,61,129,9,3,Нет влияния,2011,"районе г. Покровска,"
4,Олекминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60,23,0,120,26,0,Ослабление прочности льда,2011,"Олекминском улусе,"


In [13]:
# Потенциально можно и дальше работать с получившимся столбцом, например, попытаться вытащить название населенного
# пункта или района (через re.compile(r'[А-Я][а-я]*'), например). Однако судя по всему, так делать не стоит, так
# как названия мероприятий, из которой и сформирован новый столбец, заполнены очень неединообразно, 
# поэтому попытавшись выбрать оттуда только первые слова с заглавной буквы, можно потерять информацию. Например, 
# в тех случаях, когда мероприятие проводилось сразу в нескольких районах

# Поэтому более надежным будет проверить всё "глазами"

In [14]:
# Проверим, совпадают ли местоположение, указанное в датафрейме и то, которое мы получили через описание 
# мероприятий

# Для города Ленска
df[df['Расположение'] == 'Ленск']['Расположение_верно'].unique()

array(['Ленском районе, ', 'районе г. Ленска, ', 'Ленском улусе, ',
       'Ленском районе '], dtype=object)

In [15]:
# Ошибок нет

# Для города Покровска
df[df['Расположение'] == 'Покровск']['Расположение_верно'].unique()

array(['районе г. Покровска, ', 'Хангаласском улусе, '], dtype=object)

In [16]:
# Город Покровск - центр Хангаласского улуса. Поэтому в принципе, здесь ошибки нет, есть неточность, которую
# при необходимости можно исправить, указав везде не города, а районы (улусы)

# Для города Олекминска
df[df['Расположение'] == 'Олекминск']['Расположение_верно'].unique()

array(['Олекминском улусе, ', 'районе г. Олекминска, ',
       'Олекминском районе, ',
       'Олекминском улусе, республика Саха (Якутия)',
       'Олекминском районе ', 'Оленёкском, Анабарском районах '],
      dtype=object)

In [17]:
# Среди значений есть ошибочное: 'Оленёкском, Анабарском районах '
df[(df['Расположение'] == 'Олекминск') & (df['Расположение_верно'] == 'Оленёкском, Анабарском районах ')]

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,
109,Олекминск,СУБВ,"Определение местоположения береговой линии, гр...",71,57,22,114,4,3,Нет влияния,2020,"Оленёкском, Анабарском районах"


In [18]:
# Полное название мероприятия в этой строке
df.loc[109, 'Наименование мероприятия']

'Определение местоположения береговой линии, границ водоохранной зоны и прибрежной защитной полосы р. Анабар в Оленёкском, Анабарском районах Республики Саха (Якутия) '

In [19]:
# Действительно, здесь ошибка в указании города

# Для города Киренска
df[df['Расположение'] == 'Киренск']['Расположение_верно'].unique()

array(['пределах населенных пунктов Киренск, Кривопашкино, Петропавловск, Макарово, Алымовка, Кривая Лука Киренского района',
       'пределах населенных пунктов Киренск, Усть-Киренга, Половинка Киренского района',
       'пределах населенных пунктов Киренск, Усть-Киренга Киренского района',
       'пределах населенных пунктов Киренск, Кривошапкино, Петропавловск, Макарово, Алымовка, Кривая Лука Киренского района ',
       'пределах населенных пунктов Киренск, Кривошапкино, Петропавловск, Макарово, Алымовка, Кривая Лука Киренского района'],
      dtype=object)

In [20]:
# Здесь все верно

# Для города Сангара
df[df['Расположение'] == 'Сангар']['Расположение_верно'].unique()

array(['черте п. Сангар Кобяйского района ', 'Кобяйском улусе, ',
       'черте п. Сангар Кобяйского улуса, '], dtype=object)

In [21]:
# Сангар - административный центр Кобяйского улуса, поэтому тоже ошибки нет

# Для города Якутстка
df[df['Расположение'] == 'Якутск']['Расположение_верно'].unique()

array(['районе г. Якутска, ', 'районе г.Якутск, ', 0, 'городе Якутске", ',
       'городе Якутске ', 'районе г. Якутска,', 'городе Якутске, ',
       'районе г. Якутск, ', 'г. Якутске ',
       'Ленском, Хангаласском, Мегино-Кангаласском, Намском районах, ГО "город Якутск" ',
       'в районе г. Якутск, ',
       'ГО «Город Якутск», Горном, Намском, Хангаласском районах '],
      dtype=object)

In [22]:
# Здесь уже есть проблемы. Во-первых, есть строки, из которых не удалось отобрать название района регуляркой.
# Выведем описание этоих мероприятий

df[(df['Расположение'] == 'Якутск') & (df['Расположение_верно'] == 0)]['Наименование мероприятия'].values

array(['Капитальный ремонт гидротехнического на р.Маганка - 7км Маганского тракта, г.Якутск (Республиканская собственность)',
       'Разработка проекта "Расчистка и дноуглубление русла р. Шестаковка на территории городского округа "Город Якутск", Республика Саха (Якутия)"',
       'Определение границ водоохранной зоны и прибрежной защитной полосы озер г. Якутска, Республика Саха (Якутия)',
       'Закрепление границ водоохранной зоны и прибрежной защитной полосы озер г. Якутска, Республика Саха (Якутия)',
       'Закрепление границ водоохранной зоны и прибрежной защитной полосы озер г. Якутска, Республика Саха (Якутия)\n',
       'Определение границ водоохранной зоны и прибрежной защитной полосы р.Мархинка ГО "город Якутск" Республики Саха (Якутия)',
       'Определение границ водоохранной зоны и прибрежной защитной полосы оз. Соленое мкр. Марха г. Якутск, Республика Саха (Якутия)'],
      dtype=object)

In [23]:
# Судя по описаниям мероприятий, здесь ошибок с указанием города нет. Проблема возникла из-за другого формата строк,
# который не учитывался нашей регуляркой

# Выведем другие потенциально проблемные строки по Якутску
df[(df['Расположение'] == 'Якутск') & ((
    df['Расположение_верно'] == 'Ленском, Хангаласском, Мегино-Кангаласском, Намском районах, ГО "город Якутск" ') | (
    df['Расположение_верно'] == 'ГО «Город Якутск», Горном, Намском, Хангаласском районах '))]

,Расположение,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,
77,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,60,24,38,116,51,13,Нет влияния,2017,"Ленском, Хангаласском, Мегино-Кангаласском, На..."
107,Якутск,СУБВ,"Определение местоположения береговой линии, гр...",62,27,0,129,11,0,Нет влияния,2020,"ГО «Город Якутск», Горном, Намском, Хангаласск..."


In [24]:
# Полное описание мероприятия в этих строках
df.loc[[77, 107], 'Наименование мероприятия'].values

array(['Определение границ водоохранной зоны и прибрежной защитной полосы р.Лена в Ленском, Хангаласском, Мегино-Кангаласском, Намском районах, ГО "город Якутск" Республики Саха (Якутия)',
       'Определение местоположения береговой линии, границ водоохранной зоны и прибрежной защитной полосы р. Кэнкэмэ в ГО «Город Якутск», Горном, Намском, Хангаласском районах Республики Саха (Якутия)'],
      dtype=object)

In [25]:
# Здесь проблема заключается в том, что мероприятия проводились в рамках нескольких районов, однако в качестве 
# местоположения указан только один из них

# Наконец, проверим строки, в которых в качестве города указан субъект целиком
df[df['Расположение'] == 'Саха (Якутия)']['Расположение_верно'].unique()

array([0, 'Ленском улусе, ', 'Олекминском районе, ',
       'Олекминском улусе, ', 'Хангаласском улусе, ',
       'районе г. Якутск, ', 'Намском улусе, ', 'Кобяйском улусе, ',
       'Верхнеколымском улусе, ', 'Среднеколымском улусе, ',
       'районе г.Якутск, ',
       'местах размыва и разрушения у с. Бютяйдях Мегино-Кангаласского улуса, ',
       'районе с. Намцы Намского улуса ',
       'на р. Лена с целью разработки современных методов обеспечения безопасности жизнедеятельности на приречных территориях в период весеннего половодья'],
      dtype=object)

In [26]:
# Выведем полные названия мероприятий в этих строках
sakha = df[df['Расположение'] == 'Саха (Якутия)'][['Наименование мероприятия']]
for row in sakha.iterrows():
    print(row[0])
    print(row[1].values)
    print('--------------')

6
['Разработка проекта СКИОВО бассейна реки Лена']
--------------
23
['Разработка проекта СКИОВО бассейна реки Лена.']
--------------
26
['Исследование и оценка воздействия мероприятий по ослаблению прочности льда на прохождение весеннего половодья на реке Лена.']
--------------
29
['Ослабление прочности льда на р. Нюя в Ленском улусе, Республика Саха (Якутия)\n']
--------------
30
['Ослабление прочности льда на р.Лена в Ленском улусе, Республика Саха (Якутия)']
--------------
35
['Ослабление прочности льда на р. Нюя в Ленском улусе, Республики Саха (Якутия)']
--------------
36
['Разработка проекта "Расчистка и дноуглубление русла р.Алалайка в Олекминском районе, Республика Саха (Якутия)"']
--------------
38
['#Ослабление прочности льда на р. Лена в Ленском улусе, Республика Саха (Якутия)']
--------------
44
['Ослабление прочности льда на р. Лена в Ленском улусе, Республика Саха (Якутия)']
--------------
45
['Ослабление прочности льда на р. Лена в Олекминском улусе, Республика Саха (Як

In [27]:
# В ряде строк нет указания на конкретный район - это проблема.

# В ряде строк такое указание есть: мы вывели такие строки вместе с индексами, по которым теперь можно вставить
# конкретное расположение вместо общего "Саха (Якутск)"

In [28]:
# Создадим новый столбец для исправленных расположений и исправим ошибки в тех строках, в которых они были 
# обнаружены. В качестве нового расположения укажем информацию, найденную через описание мероприятия. Указываем
# названия административных центров районов (улусов)

# Вставляем новый столбец сразу после старого для удобства
df.insert(1, 'Расположение испр', df['Расположение'])

# Исправление ошибок по индексам
df.loc[49, 'Расположение испр'] = 'Сангар'
df.loc[[46,58], 'Расположение испр'] = 'Покровск'
df.loc[[29,30,35,38,44], 'Расположение испр'] = 'Ленск'
df.loc[[36,45], 'Расположение испр'] = 'Олекминск'
df.loc[[47,59], 'Расположение испр'] = 'Якутск'
df.loc[[36,45], 'Расположение испр'] = 'Олекминск'
df.loc[[48,60,67], 'Расположение испр'] = 'Намцы'
df.loc[50, 'Расположение испр'] = 'Зырянка'
df.loc[51, 'Расположение испр'] = 'Среднеколымск'
df.loc[[61,64], 'Расположение испр'] = 'Нижний Бестях'
df.loc[65, 'Расположение испр'] = 'Бердигестях'

# Строки с неизвестным истинным расположением, а также с мероприятиеми, проводившимися в нескольких районах. Присвоим
# в таких строках значение 0
df.loc[[77,107,109,6,23,26,76], 'Расположение испр'] = 0
df.head()

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
0,Ленск,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61,22,42,114,33,54,Ослабление прочности льда,2011,"Ленском районе,"
1,Ленск,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35,43,60,114,56,43,Нет влияния,2011,"районе г. Ленска,"
2,Якутск,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62,1,38,129,43,55,Нет влияния,2011,"районе г. Якутска,"
3,Покровск,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0,29,61,129,9,3,Нет влияния,2011,"районе г. Покровска,"
4,Олекминск,Олекминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60,23,0,120,26,0,Ослабление прочности льда,2011,"Олекминском улусе,"


In [29]:
# В столбце "влияние на водный объект" были пропущенные значения

# Посмотрим, какие названия мероприятий соотвутствуют влиянию "ослабление прочности льда" в столбце "Влияние
# на водный объект"
df['Наименование мероприятия'][df['Влияние на водный объект'] == 'Ослабление прочности льда'].unique()

array(['Ослабление прочности льда на р.Нюя в Ленском районе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р.Лена в Олекминском улусе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р.Лена в Ленском улусе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р.Лена в районе г.Якутск, Республика Саха (Якутия)',
       'Ослабление прочности льда на р. Нюя в Ленском улусе, Республика Саха (Якутия)\n',
       'Ослабление прочности льда на р.Лена в Хангаласском улусе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р.Лена в Кобяйском улусе, Республика Саха (Якутия)',
       'Ослабление прочности льда на р. Нюя в Ленском улусе, Республики Саха (Якутия)',
       '#Ослабление прочности льда на р. Лена в Ленском улусе, Республика Саха (Якутия)',
       '#Ослабление прочности льда на р. Лена в Олекминском улусе, Республика Саха (Якутия)',
       '#Ослабление прочности льда на р. Лена в Хангаласском улусе, Республика Саха (Якутия)',
  

In [30]:
# Можно сделать вывод о том, что если "Влияние..." имеет значение "Ослабление прочности льда", то же самое 
# указано и в описании мероприятия. Значит, можно заполнить эти NaN'ы с опорой на описание при помощи регулярных
# выражений по аналогии с тем, что было сделано для проверки корректного указания городов

### Основные выводы по текстовым столбцам:

* В датасете действительно есть опечатки в столбце с названиями населенных пунктов. Помимо опечаток, отсутствует единообразие ввода (*Саха(Якутия)* и *Саха Якутия* или *Олекминск* и *Олёкминск* - это одно и то же, "г." для города указан только в одном случае). Также Саха - это название региона, а всё остальное в датасете - это города


* Наиболее проблемным столбцом представляется столбец **"Описание мероприятий"**, так как он является полным описанием произведенных действий и заполнен без какого-либо единого стандарта. Как следствие, здесь очень много уникальных значений, потому что даже отличие в один знак препинания делает две совершенно одинаковые строки разными. При этом данная строка содержит ценную информацию для нас. С ней можно работать при помощи регулярных выражений


* Проблемы обнаруживается при сравнении описаний мероприятий с указанными расположениями:
    + часть мероприятий проводилась в рамках нескольких районов / городов, а в качество расположения указан только один из них. Такие мероприятия можно отмечать отдельно. Возможно, изначально отметка "*Саха (Якутия)*" в расположении предназначалась именно для таких мероприятий, однако теперь она уже не информативна
    + есть противоречия между столбцами **"Расположение"** и **"Наименование мероприятия"**: в первом указан один город, во втором другой
    + есть мероприятия без указания расположения (в датафрейме они идут как "*Саха (Якутия)*", однако по наименованию мероприятия можно восстановить конкретное название города / района.
    + также можно отметить отсутствие единообразия в указании расположений. В самом столбце **"Расположение"** указаны города, однако мероприятия проводятся не только в черте города. Поэтому, возможно, корретнее было бы указывать там районы (улусы), а не города


* Столбцы **"Тип"** и **"Влияние на водный объект"** являются категориальными, где возможны всего по два варианта: СУБВ, ГТС в типе и "Ослабление прочности льда", "Нет влияния" во влиянии. При необходимости, для дальнейшего анализа можно закодировать данную информацию (например, числовыми характеристиками 0 - нет влияния, 1 - ослабление льда)


* Также пропущенные значения в столбце **"Влияние на водный объект"** можно восстановить при помощи столбца **"Наименование мероприятия"**, так как во всех строках, где во влиянии указано *"ослабление прочности льда"*, то же самое указано и в наименовании мероприятия. Значит, при отсутствии в описании мероприятия слов *"ослабление прочности льда"* в пропущенные значения столбца **"Влияние на водный объект"** можно поставить "*нет влияния*"

### Шаг 2. Работа с ошибками числовых данных

In [31]:
# Проверим, все ли значения координат числовые: выведем все уникальные значения по столбцу "широта"
df['Широта, Градусов'].unique()

array([61, 35, 62, 0, 60, 1, 54, 59, 36, 2, 38, 7, 33, 42, 63, 13, 'НД',
       57, 29, 27, 15, 71], dtype=object)

In [32]:
# по столбцу "долгота"
df['Долгота, Градусов'].unique()

array([114, 129, 120, 115, 127, 109, 'НД', 108, 116, 122, 130, 131, 152,
       156, 119, 126, 133, 0], dtype=object)

In [33]:
# Помимо числовых, есть отметка "НД" (по-видимому, "нет данных"), при преобразовании необходимо это учитывать

# Также есть околонулевое значения широты и долготы, чего на территории Республики Саха быть не может

# Преобразуем столбцы с координатами в числовой тип. Отсутствующие значения заменяем на NaN
df.iloc[:, 4:10] = df.iloc[:, 4:10].apply(pd.to_numeric, errors='coerce')
df.head()

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
0,Ленск,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2011,"Ленском районе,"
1,Ленск,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35.0,43.0,60.0,114.0,56.0,43.0,Нет влияния,2011,"районе г. Ленска,"
2,Якутск,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62.0,1.0,38.0,129.0,43.0,55.0,Нет влияния,2011,"районе г. Якутска,"
3,Покровск,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0.0,29.0,61.0,129.0,9.0,3.0,Нет влияния,2011,"районе г. Покровска,"
4,Олекминск,Олекминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60.0,23.0,0.0,120.0,26.0,0.0,Ослабление прочности льда,2011,"Олекминском улусе,"


In [34]:
# Выведем строки с явными ошибками в значениях координат по экстремальным значениям координат, на которых 
# расположена Республика Саха
coor_problem = df[(df['Широта, Градусов'].isnull()) | (
    df['Долгота, Градусов'].isnull()) | (
    df['Широта, Градусов'] < 55) | (
    df['Широта, Градусов'] > 77) | (
    df['Долгота, Градусов'] < 106) | (
    df['Долгота, Градусов'] > 163)]

coor_problem

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
1,Ленск,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,35.0,43.0,60.0,114.0,56.0,43.0,Нет влияния,2011,"районе г. Ленска,"
3,Покровск,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0.0,29.0,61.0,129.0,9.0,3.0,Нет влияния,2011,"районе г. Покровска,"
6,Саха (Якутия),0,СУБВ,Разработка проекта СКИОВО бассейна реки Лена,0.0,2.0,62.0,129.0,44.0,0.0,Нет влияния,2011,0
9,Якутск,Якутск,СУБВ,"Разработка проекта ""Расчистка озера Сайсары в ...",1.0,1.0,62.0,129.0,41.0,37.0,Нет влияния,2011,"городе Якутске"","
10,Якутск,Якутск,СУБВ,"Разработка проекта ""Расчиста Атласовских озер ...",54.0,57.0,61.0,129.0,36.0,59.0,Нет влияния,2011,городе Якутске
12,Якутск,Якутск,СУБВ,"Разработка проекта ""Расчистка Атласовских озер...",54.0,57.0,61.0,129.0,36.0,59.0,Нет влияния,2012,городе Якутске
13,Якутск,Якутск,СУБВ,"Разработка проекта ""Расчистка озера Сайсары в ...",1.0,1.0,62.0,129.0,41.0,37.0,Нет влияния,2012,"городе Якутске"","
16,Ленск,Ленск,СУБВ,Закрепление на местности границ водоохранной ...,35.0,43.0,60.0,114.0,56.0,43.0,Нет влияния,2012,"районе г. Ленска,"
17,Олекминск,Олекминск,СУБВ,Закрепление на местности границ водоохранной ...,36.0,22.0,60.0,120.0,25.0,24.0,Нет влияния,2012,"районе г. Олекминска,"


In [35]:
# Количество таких строк
len(coor_problem)

39

In [36]:
# Можно заметить следующую закономерность: в секундах широты оказались значения около 60: возможно, произошла
# перестановка столбцов, и значения секунд на самом деле соответствуют градусам. Проверим это на строках без 
# "явных" ошибок

# Нереалистичных градусов долготы не наблюдается

In [37]:
# Строки без явных ошибок координат
df_correct = df[~df.index.isin(coor_problem.index)]
df_correct

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
0,Ленск,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2011,"Ленском районе,"
2,Якутск,Якутск,СУБВ,Определение границ водоохранной зоны и прибреж...,62.0,1.0,38.0,129.0,43.0,55.0,Нет влияния,2011,"районе г. Якутска,"
4,Олекминск,Олекминск,ГТС,Ослабление прочности льда на р.Лена в Олекминс...,60.0,23.0,0.0,120.0,26.0,0.0,Ослабление прочности льда,2011,"Олекминском улусе,"
5,Ленск,Ленск,ГТС,Ослабление прочности льда на р.Лена в Ленском ...,60.0,43.0,0.0,114.0,54.0,0.0,Ослабление прочности льда,2011,"Ленском улусе,"
7,Якутск,Якутск,ГТС,Ослабление прочности льда на р.Лена в районе г...,62.0,1.0,38.0,129.0,43.0,55.0,Ослабление прочности льда,2011,"районе г.Якутск,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Ленск,Ленск,СУБВ,Ослабление прочности льда на р. Нюя в Ленском ...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2020,"Ленском улусе,"
106,Олекминск,Олекминск,СУБВ,Ослабление прочности льда на р. Токко в Олекми...,59.0,18.0,9.0,119.0,39.0,24.0,Ослабление прочности льда,2020,"Олекминском улусе,"
107,Якутск,0,СУБВ,"Определение местоположения береговой линии, гр...",62.0,27.0,0.0,129.0,11.0,0.0,Нет влияния,2020,"ГО «Город Якутск», Горном, Намском, Хангаласск..."


In [38]:
# Действительно, значения около 60 располагаются в столбце с градусами широты

# Попробуем понять, на месте ли секунды и минуты широты, сравнив потенциально ошибочные и потенциально корректные
# координаты по одному городу

# Потенциальные ошибки по городу Покровск
coor_problem[coor_problem['Расположение испр'] == 'Покровск']

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
3,Покровск,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,0.0,29.0,61.0,129.0,9.0,3.0,Нет влияния,2011,"районе г. Покровска,"
18,Покровск,Покровск,СУБВ,Закрепление на местности границ водоохранной ...,0.0,29.0,61.0,129.0,9.0,3.0,Нет влияния,2012,"районе г. Покровска,"
46,Саха (Якутия),Покровск,ГТС,Ослабление прочности льда на р. Лена в Хангала...,29.0,15.0,61.0,130.0,10.0,8.0,Ослабление прочности льда,2015,"Хангаласском улусе,"
58,Саха (Якутия),Покровск,ГТС,Ослабление прочности льда на р.Лена в Хангалас...,29.0,15.0,61.0,130.0,10.0,8.0,Ослабление прочности льда,2016,"Хангаласском улусе,"


In [39]:
# Потенциально правильные координаты по городу Покровск
df_correct[df_correct['Расположение испр'] == 'Покровск']

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
32,Покровск,Покровск,ГТС,Ослабление прочности льда на р.Лена в Хангалас...,61.0,29.0,0.0,129.0,9.0,0.0,Ослабление прочности льда,2013,"Хангаласском улусе,"
40,Покровск,Покровск,ГТС,#Ослабление прочности льда на р. Лена в Хангал...,61.0,29.0,0.0,129.0,9.0,0.0,Ослабление прочности льда,2014,"Хангаласском улусе,"
71,Покровск,Покровск,ГТС,Ослабление прочности льда на р.Лена в Хангалас...,61.0,29.0,0.0,129.0,9.0,0.0,Ослабление прочности льда,2017,"Хангаласском улусе,"


In [40]:
# Судя по всему, перепутаны именно градусы и секунды широты. 

# Поменяем их местами в проблемных строках
coor_problem['new'] = coor_problem['Широта, Градусов']
df.loc[coor_problem.index, 'Широта, Градусов'] = coor_problem['Широта, Секунд']
df.loc[coor_problem.index, 'Широта, Секунд'] = coor_problem['new']
df.loc[coor_problem.index, :]

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
1,Ленск,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,60.0,43.0,35.0,114.0,56.0,43.0,Нет влияния,2011,"районе г. Ленска,"
3,Покровск,Покровск,СУБВ,Определение границ водоохранной зоны и прибреж...,61.0,29.0,0.0,129.0,9.0,3.0,Нет влияния,2011,"районе г. Покровска,"
6,Саха (Якутия),0,СУБВ,Разработка проекта СКИОВО бассейна реки Лена,62.0,2.0,0.0,129.0,44.0,0.0,Нет влияния,2011,0
9,Якутск,Якутск,СУБВ,"Разработка проекта ""Расчистка озера Сайсары в ...",62.0,1.0,1.0,129.0,41.0,37.0,Нет влияния,2011,"городе Якутске"","
10,Якутск,Якутск,СУБВ,"Разработка проекта ""Расчиста Атласовских озер ...",61.0,57.0,54.0,129.0,36.0,59.0,Нет влияния,2011,городе Якутске
12,Якутск,Якутск,СУБВ,"Разработка проекта ""Расчистка Атласовских озер...",61.0,57.0,54.0,129.0,36.0,59.0,Нет влияния,2012,городе Якутске
13,Якутск,Якутск,СУБВ,"Разработка проекта ""Расчистка озера Сайсары в ...",62.0,1.0,1.0,129.0,41.0,37.0,Нет влияния,2012,"городе Якутске"","
16,Ленск,Ленск,СУБВ,Закрепление на местности границ водоохранной ...,60.0,43.0,35.0,114.0,56.0,43.0,Нет влияния,2012,"районе г. Ленска,"
17,Олекминск,Олекминск,СУБВ,Закрепление на местности границ водоохранной ...,60.0,22.0,36.0,120.0,25.0,24.0,Нет влияния,2012,"районе г. Олекминска,"


In [41]:
# Также в coor_problem есть три строки с пропущенными значениями координат (NaN или 0). Выведем эти строки
df.iloc[[36,66,76],:]

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
36,Саха (Якутия),Олекминск,СУБВ,"Разработка проекта ""Расчистка и дноуглубление ...",NaN,NaN,NaN,NaN,NaN,NaN,Нет влияния,2014,"Олекминском районе,"
66,Олекминск,Олекминск,СУБВ,Расчистка и дноуглубление русла р. Алалайка в ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,"Олекминском районе,"
76,Саха (Якутия),0,СУБВ,Изучение гидроледотермических условий формиров...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2017,на р. Лена с целью разработки современных мето...


In [42]:
# Полные названия мероприятий в этих строках
df.loc[[36,66,76],'Наименование мероприятия'].values

array(['Разработка проекта "Расчистка и дноуглубление русла р.Алалайка в Олекминском районе, Республика Саха (Якутия)"',
       'Расчистка и дноуглубление русла р. Алалайка в Олекминском районе, Республики Саха (Якутия)',
       'Изучение гидроледотермических условий формирования ледовых заторов на р. Лена с целью разработки современных методов обеспечения безопасности жизнедеятельности на приречных территориях в период весеннего половодья'],
      dtype=object)

In [43]:
# Два мероприятия проводились на р. Алалайка. Возможно, в датафрейме есть другие строки с похожим мероприятием

# По третьему мероприятию невозможно установить его точное местоположение, так как ни в описании, ни в столбце 
# "Расположение" не указано точное местоположение

# Регулярное выражение для поиска
reg = re.compile('Алалайка')

search = []

# Построчная итерация датафрейма для поиска
for row in df.iterrows():
    # Запись индексов строк, если встретилось "Алалайка"
    if re.findall(reg, row[1][3]):
        search.append(row[0])

print(search)

[36, 66]


In [44]:
# Нет, больше таких мероприятий нет, а значит восстановить координаты в данных строках по ним не удастся

# Выведем строки, которые на самом деле соответствуют мероприятиям в нескольких районах. Эти строки - проблемные,
# так как судя по описанию мероприятия, оно проводилось в нескольких районах, однако при этом указаны координаты
# одной локации
df.iloc[[77,107,109],:]

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
77,Якутск,0,СУБВ,Определение границ водоохранной зоны и прибреж...,60.0,24.0,38.0,116.0,51.0,13.0,Нет влияния,2017,"Ленском, Хангаласском, Мегино-Кангаласском, На..."
107,Якутск,0,СУБВ,"Определение местоположения береговой линии, гр...",62.0,27.0,0.0,129.0,11.0,0.0,Нет влияния,2020,"ГО «Город Якутск», Горном, Намском, Хангаласск..."
109,Олекминск,0,СУБВ,"Определение местоположения береговой линии, гр...",71.0,57.0,22.0,114.0,4.0,3.0,Нет влияния,2020,"Оленёкском, Анабарском районах"


In [45]:
# Предоложительно координаты мероприятий, проводившихся в одном районе, должно плюс-минус совпадать. Проверим это

# На примере Ленска
lensk = df[df['Расположение испр'] == 'Ленск']
lensk.head()

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно
index,,,,,,,,,,,,,
0,Ленск,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2011,"Ленском районе,"
1,Ленск,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,60.0,43.0,35.0,114.0,56.0,43.0,Нет влияния,2011,"районе г. Ленска,"
5,Ленск,Ленск,ГТС,Ослабление прочности льда на р.Лена в Ленском ...,60.0,43.0,0.0,114.0,54.0,0.0,Ослабление прочности льда,2011,"Ленском улусе,"
16,Ленск,Ленск,СУБВ,Закрепление на местности границ водоохранной ...,60.0,43.0,35.0,114.0,56.0,43.0,Нет влияния,2012,"районе г. Ленска,"
20,Ленск,Ленск,ГТС,Ослабление прочности льда на р.Лена в Ленском ...,60.0,44.0,38.0,115.0,34.0,21.0,Ослабление прочности льда,2012,"Ленском улусе,"


In [46]:
# Для установления более точного местоположения выделим названия рек из описания мероприятия

# Используем две регулярки: одна для случаев, где река обозначена с пробелом (р. Лена), другая - для случаев
# без пробелов (р.Лена)
reg_river1 = re.compile(r'р\. (\w+)')
reg_river2 = re.compile(r'р\.(\w+)')

rivers = []

# построчная итерация датафрейма для работы регулярки
for row in lensk.iterrows():
    try:
        river = re.findall(reg_river1, row[1][3])
        if not river:
            river = re.findall(reg_river2, row[1][3])
        rivers.append(river[0])
    # исключение для строк, в которых регулярка вернет пустую строку
    except:
        rivers.append(0)
    
# новый столбец датафрейма
lensk['Река'] = rivers

# Посмотрим на уникальные значения получившегося столбца
lensk['Река'].unique()

array(['Нюя', 'Лена', 0], dtype=object)

In [47]:
# Проверим строку, на которой регулярка не отработала
lensk[lensk['Река'] == 0].loc[:, 'Наименование мероприятия'].values

array(['Закрепление на местности  границ водоохранной зоны прибрежной защитной полосы реки Лена в районе г. Ленска, Республика Саха (Якутия)'],
      dtype=object)

In [48]:
# Вставим значение "Лена" вместо 0
lensk.loc[lensk['Река'] == 0, 'Река'] = 'Лена'

In [49]:
# Выведем строки по реке Нюя
lensk[lensk['Река'] == 'Нюя']

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно,Река
index,,,,,,,,,,,,,,
0,Ленск,Ленск,СУБВ,Ослабление прочности льда на р.Нюя в Ленском р...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2011,"Ленском районе,",Нюя
29,Саха (Якутия),Ленск,СУБВ,Ослабление прочности льда на р. Нюя в Ленском ...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2013,"Ленском улусе,",Нюя
35,Саха (Якутия),Ленск,СУБВ,Ослабление прочности льда на р. Нюя в Ленском ...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2014,"Ленском улусе,",Нюя
52,Ленск,Ленск,ГТС,Ослабление прочности льда на р. Нюя в Ленском ...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2015,Ленском районе,Нюя
62,Ленск,Ленск,ГТС,Ослабление прочности льда на р. Нюя в Ленском ...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2016,"Ленском улусе,",Нюя
74,Ленск,Ленск,ГТС,Ослабление прочности льда на р.Нюя в Ленском у...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2017,"Ленском улусе,",Нюя
86,Ленск,Ленск,СУБВ,Ослабление прочности льда на р. Нюя в Ленском ...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2018,"Ленском улусе,",Нюя
105,Ленск,Ленск,СУБВ,Ослабление прочности льда на р. Нюя в Ленском ...,61.0,22.0,42.0,114.0,33.0,54.0,Ослабление прочности льда,2020,"Ленском улусе,",Нюя


In [50]:
# Выведем строки по реке Лена
lensk[lensk['Река'] == 'Лена']

,Расположение,Расположение испр,Тип,Наименование мероприятия,"Широта, Градусов","Широта, Минут","Широта, Секунд","Долгота, Градусов","Долгота, Минут","Долгота, Секунд",Влияние на водный объект,Год,Расположение_верно,Река
index,,,,,,,,,,,,,,
1,Ленск,Ленск,СУБВ,Определение границ водоохранной зоны и прибреж...,60.0,43.0,35.0,114.0,56.0,43.0,Нет влияния,2011,"районе г. Ленска,",Лена
5,Ленск,Ленск,ГТС,Ослабление прочности льда на р.Лена в Ленском ...,60.0,43.0,0.0,114.0,54.0,0.0,Ослабление прочности льда,2011,"Ленском улусе,",Лена
16,Ленск,Ленск,СУБВ,Закрепление на местности границ водоохранной ...,60.0,43.0,35.0,114.0,56.0,43.0,Нет влияния,2012,"районе г. Ленска,",Лена
20,Ленск,Ленск,ГТС,Ослабление прочности льда на р.Лена в Ленском ...,60.0,44.0,38.0,115.0,34.0,21.0,Ослабление прочности льда,2012,"Ленском улусе,",Лена
30,Саха (Якутия),Ленск,ГТС,Ослабление прочности льда на р.Лена в Ленском ...,60.0,8.0,55.0,109.0,41.0,2.0,Ослабление прочности льда,2013,"Ленском улусе,",Лена
38,Саха (Якутия),Ленск,ГТС,#Ослабление прочности льда на р. Лена в Ленско...,60.0,8.0,55.0,109.0,41.0,2.0,Ослабление прочности льда,2014,"Ленском улусе,",Лена
44,Саха (Якутия),Ленск,ГТС,Ослабление прочности льда на р. Лена в Ленском...,60.0,44.0,38.0,116.0,56.0,55.0,Ослабление прочности льда,2015,"Ленском улусе,",Лена
56,Ленск,Ленск,ГТС,Ослабление прочности льда на р.Лена в Ленском ...,60.0,43.0,0.0,114.0,54.0,0.0,Ослабление прочности льда,2016,"Ленском улусе,",Лена
69,Ленск,Ленск,ГТС,Ослабление прочности льда на р.Лена в Ленском ...,60.0,43.0,0.0,114.0,54.0,0.0,Ослабление прочности льда,2016,"Ленском улусе,",Лена


In [49]:
# Потенциально проблемными могут быть строки, в которых значения координат отличаются от остальных. Так, в строках
# с индексами 30 и 38 существует вероятность того, что перепутаны местами минуты и секунды широты, так как минут
# с таким маленьким значением больше ни в каких строках не наблюдается

### Выводы по числовым данным

* В датафрейме присутствуют ошибки ввода координат. Наиболее явная ошибка - отсутствие значений. В исходном датафрейме они указаны как "*НД*". Также ошибкой являются значения, не соответствующие координатам расположения Республики Саха: 55 градусов широты на юге, 77 - на севере, 106 градусов долготы на востоке и 163 - на западе.


* Часть координат удалось исправить: в них произошла перестановка столбцов градусов широты и секунд широты.


* Также остается проблема единообразия точности координат. Например, в датафрейме есть строки с нулевым значением *секунды* широты и долготы. Это может свидетельствовать не только о том, что на данной территории действительно такая координата, но также и о том, что некоторые территории предоставляют неполную информацию о координатах


* В проверке корректности координат может помочь столбец с описанием мероприятий, из которого можно вытащить уточненное местоположение, а также название реки, на которой проводилось мероприятие. Предполагается, что мероприятия, проводившиеся в рамках одного района (улуса) на одной реке, должны иметь примерно одинаковые координаты. В датафрейме есть строки, в которых это не соблюдается (например, строки с индексами 30 и 38 для Ленска и Ленского района). Такие строки потенциально содержат ошибки.


* В датафрейме есть три строки с отсутствующими координатами, также есть мероприятия, по которым невозможно установить их точное местоположение по описанию. Проверка координат таких мероприятий крайне затруднительна без привлечения дополнительной информации.


* Также затруднительно установление координат для тех мероприятий, которые проводились сразу в нескольких районах